In [14]:
import numpy as np
from pickle import load, dump
import os

# Add other service level measures



In [12]:
def import_results(exp, ab, beta, e, strat):

    if exp == 1: tit = "Age_service_level"
    elif exp == 2: tit = "TP_service_level"
    elif exp == 3: tit = "Waste_control"

    file = open(f"../Experiments/Results/T30_S150_n4/{tit}_{ab}_{beta}_{e}_stat-{strat}", "rb")
    results = load(file); file.close()

    return results

def import_performance_metrics(exp, ab, beta, e, strat):

    if exp == 1: tit = "Age_service_level"
    elif exp == 2: tit = "TP_service_level"
    elif exp == 3: tit = "Waste_control"

    file = open(f"../Experiments/Performance metrics/T30_S150_n4/{tit}_{ab}_{beta}_{e}_stat-{strat}", "rb")
    metrics = load(file); file.close()

    return metrics

def export_performance_metrics(tup, exp, ab, beta, e, strat):

    if exp == 1: tit = "Age_service_level"
    elif exp == 2: tit = "TP_service_level"
    elif exp == 3: tit = "Waste_control"

    new_dir = f"../Experiments/Performance metrics/T30_S150_n4/"
    if not os.path.exists(new_dir): os.makedirs(new_dir)
    file = open(new_dir + f"{tit}_{ab}_{beta}_{e}_stat-{strat}", "wb")
    dump(tup, file); file.close()

def import_global_parameters_and_demand(T,S,n):

    file = open(f"../Experiments/Parameters/Global_T{T}_S{S}_n{n}", "rb")
    params = load(file); file.close()

    file = open(f"../Experiments/Parameters/Demand_T{T}_S{S}_n{n}", "rb")
    d = load(file); file.close()

    return params, d

T = 30; S = 150; n = 4; reps = 5
a0 = [0.4]; an1 = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
all_betas = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
betas = {b:[beta for beta in all_betas if beta <= b] for b in an1}; epsilons = [1]

(T, S, G, n, h, c, f, C), d = import_global_parameters_and_demand(T, S, n)
I0 = {g:0 for g in G[:-1]}; phi = {s:1/len(S) for s in S}
alpha = {(a,b):{g:a if g < G[-1] else b for g in G} for a in a0 for b in an1 if a <= b}

In [13]:
for strat in ["stat","dyn"]:
    for exp in [2]:
        for (a,b) in alpha:
            for beta in betas[b]:
                for e in epsilons:
                    
                    y, x, z, l, w, I = import_results(exp, (a,b), beta, e, strat)
                    holding, production, setup, total, period_sl, total_sl, waste_prod, waste_dem = import_performance_metrics(exp, (a,b), beta, e, strat)
                    
                    lit_period_sl = {ix:{(t,g):sum(phi[s]*sum(z[ix][s][t,k] for k in range(g+1)) for s in S)/sum(phi[s]*d[ix][s][t] for s in S) for t in T for g in G} for ix in range(reps)}
                    lit_total_sl = {ix:{g:sum(phi[s]*sum(z[ix][s][t,k] for t in T for k in range(g+1)) for s in S)/sum(phi[s]*sum(d[ix][s][t] for t in T) for s in S) for g in G} for ix in range(reps)}
                    
                    export_performance_metrics((holding, production, setup, total, period_sl, total_sl, waste_prod, waste_dem, lit_period_sl, lit_total_sl), exp, (a,b), beta, e, strat)


# Change names


In [19]:
T = 30; S = 150; n = 4; reps = 5
#a0 = [0.4, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]; an1 = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
a0 = [0.4]; an1 = [0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
all_betas = [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.975]
betas = {b:[beta for beta in all_betas if beta <= b] for b in an1}; epsilons = [1]
#betas = {b:[0] for b in an1}; epsilons = [1]

alpha = {(a,b):{g:a if g < G[-1] else b for g in G} for a in a0 for b in an1 if a <= b}

In [20]:

for folder in ["Results", "Performance metrics"]:
    for strat in ["stat","dyn"]:
        for (a,b) in alpha:
            for beta in betas[b]:
                for e in epsilons:

                    old_file = os.path.join(f"../Experiments/{folder}/T{T}_S{S}_n{n}/", f"TP_service_level_{a,b}_{beta}_{e}_stat-{strat}")
                    new_file = os.path.join(f"../Experiments/{folder}/T{T}_S{S}_n{n}/", f"TP_service_level_{a,b}_{beta}_{e}_stat-{strat}_base")
                    
                    os.rename(old_file, new_file)


# Demand break for FIFO stat-stat

In [7]:
file = open("C:/Users/ari_r/OneDrive - Universidad de los andes/Montreal 2024-10/stoch_LSP_PP/Experiments/Parameters/Demand_T20_S150_n4", "rb")
d = load(file); file.close()

file = open("C:/Users/ari_r/OneDrive - Universidad de los andes/Montreal 2024-10/stoch_LSP_PP/Experiments/Parameters/Global_T20_S150_n4", "rb")
(T, S, G, n, h, c, f, C) = load(file); file.close()

In [10]:
compl_scenarios = [d[ix][s] for ix in d for s in S]
compl_scenarios = compl_scenarios[:600]

In [16]:
scenarios = 20; reps = 30

new_demand = {ix:{(s+1):compl_scenarios[ix*scenarios+s] for s in range(scenarios)} for ix in range(reps)}

In [20]:
file = open("C:/Users/ari_r/OneDrive - Universidad de los andes/Montreal 2024-10/stoch_LSP_PP/Experiments/Parameters/Demand_T20_S150_n4_FIFO", "wb")
dump(new_demand,file); file.close()
